In [7]:
from pyannote.audio import Pipeline
import os
from dotenv import load_dotenv
load_dotenv() 
from pydub import AudioSegment
import random
import torch

In [10]:
load_dotenv() 
token = os.getenv("HUGGING_FACE_TOKEN")
file_path_audio = "test_12.wav"

In [11]:
def cut_audio_segment(input_path: str, start_sec: float, end_sec: float, output_path: str = None) -> AudioSegment:
    audio = AudioSegment.from_file(input_path)
    start_ms = int(start_sec * 1000)
    end_ms = int(end_sec * 1000)
    cut_audio = audio[start_ms:end_ms]
    if output_path:
        cut_audio.export(output_path, format="mp3")

    return cut_audio

def function_allocate_with_successive(list_timestamp):
  os.mkdir('speakers')
  unique_elements = set([ f[2] for f in list_timestamp])
  no_speakers = len(unique_elements)
  print(f"Number of speakers: {no_speakers}")
  for speaker in unique_elements:
    os.mkdir('speakers/'+speaker)

  start = 0
  end = 0
  previous_speaker = None
  for timestamp in list_timestamp:
    duration = timestamp[1]-timestamp[0]
    if duration<2:
      continue
    if previous_speaker == timestamp[2]:
      end = timestamp[1]
    else:
      if previous_speaker:
        cut_audio_segment('all_audios.mp3',start,end,'speakers/'+previous_speaker+'/'+f"{start:.1f}_{end:.1f}.mp3")
      start = timestamp[0]
      end = timestamp[1]
      previous_speaker = timestamp[2]
  cut_audio_segment('all_audios.mp3',start,end,'speakers/'+previous_speaker+'/'+f"{start:.1f}_{end:.1f}.mp3")

def function_allocate(list_timestamp, audio_path):
  os.mkdir('speakers')
  unique_elements = set([ f[2] for f in list_timestamp])
  no_speakers = len(unique_elements)
  print(f"Number of speakers: {no_speakers}")
  for speaker in unique_elements:
    os.mkdir('speakers/'+speaker)


  for timestamp in list_timestamp:
    start = timestamp[0]
    end = timestamp[1]
    speaker = timestamp[2]

    duration = timestamp[1]-timestamp[0]
    if duration<1:
      continue

    cut_audio_segment(audio_path,start,end,'speakers/'+speaker+'/'+f"{start:.1f}_{end:.1f}.mp3")

def function_list_timestamp(audio_path):
  pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=token)

  pipeline.to(torch.device("cuda"))

  diarization = pipeline(audio_path)

  list_timestamp = []
  for turn, _, speaker in diarization.itertracks(yield_label=True):
      #print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
      list_timestamp.append((turn.start,turn.end,speaker))
  return list_timestamp

def present_list_timestamps(list_timestamp):
  for f in list_timestamp:
    if f[1]-f[0]>=1:
      m1 = str(int(f[0]/60))
      s1 = str(int(f[0]%60))
      m2 = str(int(f[1]/60))
      s2 = str(int(f[1]%60))
      first_t = m1+":"+s1 if int(s1)>=10 else m1+":"+"0"+s1
      second_t = m2+":"+s2 if int(s2)>=10 else m2+":"+"0"+s2
      print((first_t,second_t,f[2]))


In [15]:
list_timestamp = function_list_timestamp(file_path_audio)
present_list_timestamps(list_timestamp)
function_allocate(list_timestamp,file_path_audio)

/home/lenovo/Desktop/Ognjen/ClickerProject/venv/lib/python3.12/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


('0:00', '0:01', 'SPEAKER_00')
('0:01', '0:03', 'SPEAKER_02')
('0:03', '0:06', 'SPEAKER_00')
('0:05', '0:08', 'SPEAKER_02')
('0:08', '0:11', 'SPEAKER_00')
('0:11', '0:12', 'SPEAKER_01')
('0:13', '0:14', 'SPEAKER_02')
Number of speakers: 3
